In [1]:
from tqdm import tqdm
from glob import glob
import pandas as pd
import whisper

c:\Users\snell\anaconda3\envs\thesis_env\lib\site-packages\whisper\timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


# Loading files

In [2]:
DATASET_PATH = '../data/audios/'
DATASET_FILES = glob(DATASET_PATH + '*.mp3')

# Using Whisper

In [3]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [02:55<00:00, 8.72MiB/s]


In [4]:
final_dataset = []

for audio_file in tqdm(DATASET_FILES):
    audio_data = {}
    audio_path = audio_file
    audio_id = audio_path.replace(DATASET_PATH, '').replace('.mp3', '')
    audio_id = audio_id.split('\\')[-1]  # Extract numbers after final '\'
    
    audio = whisper.load_audio(audio_file)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    _, probs = model.detect_language(mel)
    detected_language = max(probs, key=probs.get)
    
    result = model.transcribe(audio_path)
    
    audio_data['audio_id'] = audio_id
    audio_data['transcription'] = result["text"].strip()
    audio_data['detected_language'] = detected_language
    
    final_dataset.append(audio_data)
    
final_dataset_df = pd.DataFrame.from_dict(final_dataset)
final_dataset_df.to_csv('transcripted_audios.csv', index=False)

  0%|          | 0/1185 [00:00<?, ?it/s]c:\Users\snell\anaconda3\envs\thesis_env\lib\site-packages\whisper\transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
100%|██████████| 1185/1185 [12:27:08<00:00, 37.83s/it]   
